# Ciencia de Datos I<a class="tocSkip">
## Electiva Procesamiento de Señales I <a class="tocSkip">
### Ingenieria Electrónica <a class="tocSkip">
### Universidad Popular del Cesar <a class="tocSkip">
### Prof.: Jose Ramón Iglesias Gamarra - [https://github.com/joseramoniglesias/](https://github.com/joseramoniglesias/) <a class="tocSkip">
  **joseiglesias@unicesar.edu.co**



# IMPORTANTE:

En este código por favor:

*   No agregar celdas nuevas
*   No agregar librerías nuevas
*   No usar otro nombre de variables
*   No cambiar el clasificador ni el número de vecinos del KNN
*   No hacer gráficas


<font color='red'>Esta tarea será corregida automáticamente, y si no se cumple alguna de las condiciones anteriores, es posible que la tarea no se corrija correctamente.</font>

Si quieres hacer modificaciones y probar estrategias distintas, te invito a hacerlo en otro código (no en éste). <font color='red'>Este código debe modificarse sólo en los lugares indicados.</font>

<font color='orange'> Evaluación:</font>
 Este ejercicio tiene 20 ítems a evaluar, denotados como Q00, Q01, ... Q19 a lo largo del notebook como `<Evaluación Qi>`. En tu evaluación se revisará cómo han sido cada uno de estos 20 items. Tu nota en este ejercicio será el número de ítems correctamente hechos dividido por 20 multiplicado por 100 (en escala de 0 a 100%).


# Ejercicio 04: Detección de Caras usando Características de Textura
----
ENTREGA: Se realiza en la clase, <font color='red'>el nombre del archivo debe coincidir con tu número de alumno/a)</font>.

----
En este ejercicio se realizará la clasificación entre dos clases ([ver base de datos](https://www.dropbox.com/sh/40903rl8lsajfwj/AAD3XPQ1qE2c9WBRTFN5IIeMa?dl=0)):

* Clase 0: Caras (contiene fotos tomadas de caras frontales y bien alineadas)

* Clase 1: No-caras (contiene fotos de objetos que no son caras, por ejemplo sillas, mesas, plantas, etc.)

La base de datos contiene 240 imágenes de 80x80 pixeles (120 imágenes por clase).

**ADVERTENCIA:** Este ejercicio tiene fines pedagógicos sólamente, con la idea de que la solución a este problema pueda ejecutarse en un par de minutos. Un buen detector de caras sigue esta idea pero con miles de imágenes por clase.

A continuación, en cada PREGUNTA deberás ingresar alguna(s) líneas de código, o completar donde aparezca '<= COMPLETAR AQUI".




# PREGUNTA 0: Cambiar nombre de archivo

Cambia el nombre del archivo de este código de tal forma que sea tu número de alumno/a. Por ejemplo, si tu número de alumno/a es '1234567J' entonces este Notebook debe llamarse '1234567J.ipynb'. Para hacerlo debes hacer un click con el mouse sobre el nombre del archivo (arriba a la izquierda) y renombrar el nombre del archivo.

In [3]:
Nombre_de_archivo_cambiado =  # <== COMPLETAR AQUI ESCRIBIENDO EL NÚMERO 1 CUANDO HAYAS CAMBIADO EL NOMBRE

SyntaxError: invalid syntax (3917752621.py, line 1)

# PREGUNTA 1: Datos personales

Ingresa tus datos para poder evaluarte, 
<font color='red'>si tus datos son incorrectos, este ejercicio no podrá ser corregido y por lo tanto no tendrás nota en esta actividad.</font>

In [ ]:
NumeroID         = ''   # <= COMPLETAR AQUI NUMERO DE ALUMNO/A ENTRE APOSTROFES
PrimerNombre     = ''   # <= COMPLETAR AQUI TU PRIMER NOMBRE
PrimerApellido   = ''   # <= COMPLETAR AQUI TU PRIMER APELLIDO
Correo           = ''   # <= COMPLETAR AQUI TU CORRECO ELECTRONICO
Datos_ingresados =      # <= COMPLETAR AQUI ESCRIBIENDO 1 CUANDO HAYAS INGRESADO TUS DATOS

stid             = PrimerNombre+' '+PrimerApellido+' (ID:'+NumeroID+', email:'+Correo+'):'
print('Registro: '+stid)

## Setup de librerías

In [4]:
#
# EJECUTAR ESTA CELDA SIN MODIFICARLA
#
import cv2
import numpy as np
import matplotlib.pyplot as plt
from   sklearn.metrics import confusion_matrix, accuracy_score
from   sklearn.neighbors import KNeighborsClassifier
from   seaborn import heatmap
from   tqdm.auto import tqdm
import skimage
from   skimage.filters import gabor_kernel
from   scipy import ndimage as ndi
from   skimage.feature import graycoprops,graycomatrix,local_binary_pattern

print('Librerías cargadas.')


Librerías cargadas.


## Carga de base de datos

La base de datos consiste en 2 clases y 120 imágenes por clase. Se almacenan en la carpeta facedetection con el formato face_xx_nnn.png, donde xx es el ID de la clase (01 para caras, 02 para no caras) y nnn es el número de la foto de la clase (01, 02, ... 120). Las fotos son de 80x80 pixeles.

In [6]:
#
# EJECUTAR ESTA CELDA SIN MODIFICARLA
#

# Carga de base de datos
!wget https://www.dropbox.com/scl/fi/i4x4s3aicy8fb0rmzv26v/facedetection.zip?rlkey=yto4omy3x07qnp8yjd58uinms&dl=0
!unzip facedetection
print('Base de datos cargada.')

"wget" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.
"dl" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


Base de datos cargada.


"unzip" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


## Funciones necesarias

In [ ]:
#
# EJECUTAR ESTA CELDA SIN MODIFICARLA
#

def num2fixstr(x,d):
  # example num2fixstr(2,5) returns '00002'
  # example num2fixstr(19,3) returns '019'
  st = '%0*d' % (d,x)
  return st

def ImageLoad(prefix,num_char,num_img,echo='off'):
  st   = prefix + num2fixstr(num_char,2) + '_' + num2fixstr(num_img,3) + '.png'
  if echo == 'on':
    print('loading image '+st+'...')
  img    = plt.imread(st)
  return img

def MatrixStats(X):
  p = np.zeros((6),'int')
  p[0] = X.shape[0]
  p[1] = X.shape[1]
  p[2] = X.min()
  p[3] = X.max()
  p[4] = X.mean()
  p[5] = X.std()
  return p
  
def CheckMatrix(st,X,ps):
  p = MatrixStats(X)
  d = p-ps
  t = not(np.sum(d)==0)
  if t:
    print(st + ': Revisar, esta matriz presenta errores...')
  else:
    print(st + ': ok')
  return t

def CheckPrint(Y,st=True):
    
  if np.sum(Y)>0:
    if st:
      print('Revisar, hay error en al menos uno de estos cálculos, no continuar con las siguientes celdas hasta no corregir este error.')
    else:
      print('Revisar, hay error en al menos uno de los datos personales (no coinciden con los ingresados en la PREGUNTA 1),')
      print('NO SUBIR TAREA ANTES DE CORREGIR ESTE ERROR.')
      print('(tu ejercicio corre el riesgo de no ser corregido)')
  else:
    if st:
      print('Felicitaciones! Los cálculos han sido realizados correctamente.')
    else:
      print('Felicitaciones! ahora puedes subir este archivo a Google Classroom.')
  
chk = [[240,59,0,2920,108,157],[240,28,0,57946,171,1406],[240,67,0,794,43,68],[200,59,0,2920,108,154],[40,59,0,1909,108,172],[2,2,0,20,10,8],[200,28,0,57946,188,1530],[40,28,0,2973,86,373],[2,2,1,19,10,7],[200,67,0,794,44,70],[40,67,0,405,39,59],[2,2,2,18,10,7],[200,154,0,57946,95,663],[40,154,0,2973,74,198],[2,2,0,20,10,7],[200,5,0,575,51,81],[40,5,0,430,53,87],[2,2,0,20,10,9]]

# Separación entre training y testing
def SplitTrainTest(X,y,n):

  K      = np.max(y)+1              # número de clases
  N      = np.int(X.shape[0]/K)     # numeros de muestras por clase
  Ntrain = n*K                      # número de muestras para el training
  Ntest  = K*N-Ntrain               # número de muestras para el testing
  M      = X.shape[1]               # número de características por muestra
  Xtrain = np.zeros((Ntrain,M))     # subset de training
  ytrain = np.zeros((Ntrain),'int') # ground truth del training         
  Xtest  = np.zeros((Ntest,M))      # subset de testing
  ytest  = np.zeros((Ntest),'int')  # ground truth del testing  

  # contadores
  itrain = 0
  itest  = 0
  t      = 0

  for j in range(K):     # para cada clase
    for i in range(N):   # para cada imagen de la clase
      if i<n: # training
        Xtrain[itrain,:] = X[t,:]
        ytrain[itrain] = y[t]
        itrain = itrain+1
      else:  # testing
        Xtest[itest,:] = X[t,:]
        ytest[itest] = y[t]
        itest = itest+1
      t = t+1
  
  return Xtrain,ytrain,Xtest,ytest

# Clasificación usando KNN con 3 vecinos
def ClassifierKNN(Xtrain,ytrain,Xtest,ytest):
  Xtrain, a, b = normalize(Xtrain)
  Xtest        = Xtest * a + b
  knn = KNeighborsClassifier(n_neighbors=3)
  knn.fit(Xtrain, ytrain)
  ypred        = knn.predict(Xtest)
  acc          = accuracy_score(ytest,ypred)
  print('Entrenando con '+str(Xtrain.shape[0])+' muestras y probando con '+str(Xtest.shape[0])+' muestras')
  print('Testing Accuracy = '+str(acc*100)+'%')
  C = confusion_matrix(ytest,ypred)
  print('Matriz de Confusión:')
  print(C)
  return acc,C

def lbp(img,hdiv=1, vdiv=1, mapping='nri_uniform',norm=False,names=False):
  if mapping == 'nri_uniform':
    n_bins = 59
    st     = 'LBP'
  else:
    n_bins = 10
    st     = 'LBPri'
  (nv,nh) = (vdiv,hdiv)
  nn  = int(np.fix(img.shape[0]/nv))
  mm  = int(np.fix(img.shape[1]/nh))
  k = 0
  for r in range(0,img.shape[0] - nn+1, nn):
    for c in range(0,img.shape[1] - mm+1, mm):
      w = img[r:r+nn,c:c+mm]
      lbp = local_binary_pattern(w,8,1,mapping)
      (xrc, _) = np.histogram(lbp.ravel(), bins=n_bins,range=(0, n_bins))
      if k==0:
        X = xrc
        k = 1
      else:
        X = np.concatenate((X,xrc))
  if norm:
    X = X/np.linalg.norm(X)
  if names==True:
    Xn = []
    for i in range(vdiv):
      for j in range(hdiv):
        for k in range(n_bins):
          Xn.append(st+'('+str(i)+','+str(j)+')-'+str(k))
    return X,Xn
  else:
    return X


def imageload(prefix,num_class,digits_class,num_img,digits_img,sep='_',echo='off'):
  st   = prefix + num2fixstr(num_class,digits_class) + sep + num2fixstr(num_img,digits_img) + '.png'
  if echo == 'on':
    print('loading image '+st+'...')
  img    = plt.imread(st)
  return img

def haralick(img,hdiv=1, vdiv=1, distance=1,norm=False,names=False):
  (nv,nh) = (vdiv,hdiv)
  nn  = int(np.fix(img.shape[0]/nv))
  mm  = int(np.fix(img.shape[1]/nh))
  k = 0
  fst = ['contrast','dissimilarity','homogeneity','energy','correlation','ASM']
  for r in range(0,img.shape[0] - nn+1, nn):
    for c in range(0,img.shape[1] - mm+1, mm):
      w  = img[r:r+nn,c:c+mm]
      g  = skimage.feature.graycomatrix(w.astype(int), [distance], [0, np.pi/4, np.pi/2, 3*np.pi/4], levels=256)
      x0 = skimage.feature.graycoprops(g, fst[0])
      x1 = skimage.feature.graycoprops(g, fst[1])
      x2 = skimage.feature.graycoprops(g, fst[2])
      x3 = skimage.feature.graycoprops(g, fst[3])
      x4 = skimage.feature.graycoprops(g, fst[4])
      x5 = skimage.feature.graycoprops(g, fst[5])
      #haralick = [np.mean(x1),np.mean(x2),np.mean(x3),np.mean(x4),np.mean(x5),np.mean(x6)]
      haralick = np.concatenate((x0,x1,x2,x3,x4,x5), axis=1)
      if k==0:
        X = haralick[0]
        k = 1
      else:
        X = np.concatenate((X,haralick))
  if norm:
    X = X/np.linalg.norm(X)

  if names==True:
    Xn = []
    for i in range(vdiv):
      for j in range(hdiv):
        for k in range(6):
          Xn.append('Haralick('+str(i)+','+str(j)+')-'+fst[k])
    return X,Xn
  else:
    return X

def gabor(img,hdiv=1, vdiv=1,angles=4,sigmas=(1,3),frequencies=(0.05, 0.25),norm=False):
  # from scipy
  kernels = []
  for theta in range(angles):
    theta = theta / 4. * np.pi
    for sigma in sigmas:
      for frequency in frequencies:
        kernel = np.real(gabor_kernel(frequency,  theta=theta,
                                      sigma_x=sigma, sigma_y=sigma))
        kernels.append(kernel)
  n       = len(kernels)
  (nv,nh) = (vdiv,hdiv)
  nn  = int(np.fix(img.shape[0]/nv))
  mm  = int(np.fix(img.shape[1]/nh))
  k = 0
  feats = np.zeros((len(kernels), 2), dtype=np.double)
  X = np.zeros((n*nh*nv*2,))
  t = 0
  for r in range(0,img.shape[0] - nn+1, nn):
    for c in range(0,img.shape[1] - mm+1, mm):
      w  = img[r:r+nn,c:c+mm]
      for j in range(n): #,kernel in enumerate(kernels):
        kernel = kernels[j]
        filtered = ndi.convolve(w, kernel, mode='wrap')
        X[t]   = filtered.mean()
        X[t+1] = filtered.var()
        t = t+2
  if norm:
    X = X/np.linalg.norm(X)
  return X

def minmax(X):
  ma = X.max(0)
  mi = X.min(0)
  md    = ma-mi
  a     = 1/ md
  b     = -mi/md
  return X * a + b, a, b

# PREGUNTA 2: Extracción de características

En esta pregunta se debe realizar la extracción de características de la imagen `img`. Las características a extraer son las basadas en LBP, Haralick y Gabor y se extraen usando los siguientes comandos de la librería PyBalu:

## 1) Cómo extraer características LBP clásico:

Con el comando

`f  = lbp(img, hdiv=a, vdiv=b, mapping='nri_uniform')`

se divide la imagen `img` en `a x b` particiones y en cada una de ellas se extrae el descriptor LBP (vector de 59 elementos). En este caso `f` es un vector de `59*a*b` elementos. En este ejercicio trabajaremos con `a = b = 1`, es decir sólo una partición por imagen.

## 2) Cómo extraer características de Haralick:

Con el comando

`f = haralick(img, distance=d)`

se extrae el vector Haralick para una distancia de `d` pixeles. En este caso `f` es un vector de 28 elementos (independiente de la distancia `d`). En este ejercicio trabajaremos con `d = 3`.


## 3) Cómo extraer características de Gabor:

Con el comando

`f = gabor(img, rotations=r, dilations=s)`

se extrae el vector Gabor para `r` rotaciones y `s` escalas. En este caso `f` es un vector de `r*s+3` elementos. En este ejercicio trabajaremos con `r = s = 8`.

In [ ]:
print('Ejecución de '+stid)

K    = ???   # <= COMPLETAR AQUI EL NUMERO DE CLASES
N    = ???   # <= COMPLETAR AQUI EL NUMERO DE IMAGENES POR CLASE
Mlbp = ???   # <= COMPLETAR AQUI EL NUMERO DE ELEMENTOS DEL VECTOR LBP
Mhar = ???   # <= COMPLETAR AQUI EL NUMERO DE ELEMENTOS DEL VECTOR DE HARALICK
Mgab = ???   # <= COMPLETAR AQUI EL NUMERO DE ELEMENTOS DEL VECTOR DE GABOR

Xlbp  = np.zeros((K*N,Mlbp))    # K*N muestras (filas), y Mlbp características (columnas)
Xhar  = np.zeros((K*N,Mhar))    # K*N muestras (filas), y Mhar características (columnas)
Xgab  = np.zeros((K*N,Mgab))    # K*N muestras (filas), y Mgab características (columnas)

y     = np.zeros((K*N),'int')   # ground truth (clasificacion ideal)

t = 0
print('Cargando imagenes y extrayendo características...')
for j in range(K):            # para cada clase
  print('Extrayendo características: batch '+str(j+1)+'/'+str(K))
  for i in tqdm(range(N)):                # para cada imagen de la clase
    # Lectura de la imagen
    img    = 255*ImageLoad('facedetection/face_',j+1,i+1,echo='off')

    # Extracción de características

    # LBP clásico de 1x1 particiones
    Xlbp[t,:] = ??? # <= COMPLETAR AQUI PARA EXTRAER LBP (VER HELP MAS ARRIBA)

    # Haralick con d = 3 pixeles
    Xhar[t,:] = ??? # <= COMPLETAR AQUI PARA EXTRAER HARALICK (VER HELP MAS ARRIBA)

    # Gabor con 8 rotaciones y 8 escalas
    Xgab[t,:] = ??? # <= COMPLETAR AQUI PARA EXTRAER GABOR (VER HELP MAS ARRIBA)


    y[t] = j # la muestra t pertenece a la clase j
    t = t+1

In [ ]:
#
# EJECUTAR ESTA CELDA PARA EVALUAR LA EXTRACCiÓN DE CARACTERÍSTICAS
#

T = np.zeros((20),'int')
T[0] = CheckMatrix('Xlbp',Xlbp,chk[0]) # <Evaluación Q00>
T[1] = CheckMatrix('Xhar',Xhar,chk[1]) # <Evaluación Q01>
T[2] = CheckMatrix('Xgab',Xgab,chk[2]) # <Evaluación Q02>

CheckPrint(T[0:3])


# PREGUNTA 3: Clasificación usando LBP

En la pregunta anterior, fueron extraídas las características LBP, Haralick y Gabor para todas las imágenes de la base de datos. No olvidar que son 120 imágenes por clase, y que son 2 clases. 

A continuación haremos como ejemplo un clasificador de vecinos más cercanos. 

<font color='orange'>**IMPORTANTE:** En todo el Ejercicio, el clasificador será SIEMPRE un KNN de 3 vecinos que será entrenado (training) con las primeras 100 muestras de cada clase, y probado (testing) con las 20 restantes.


Los pasos a seguir para este ejercicio son los siguientes tres:

## Primer paso: Cargar datos 

En este paso, cargamos en la matriz `X` las características extrídas que van a ser usadas por el clasificador. En nuestro primer ejemplo serían las caractererísticas LBP, es decir:

`X = Xlbp`

(el vector `y`, que contiene la clase a la que pertenece cada clase, es el mismo para LBP, Haralick y Gabor, así es que ya está definido en la pregunta anterior)

## Segundo paso: Separar Training/Testing

En este paso, separamos el conjunto de datos en training y testing usando el comando:

`Xtrain,ytrain,Xtest,ytest = SplitTrainTest(X,y,ntrain)`

donde `ntrain` es el número de muestras por clase a usar en el conjunto de training. 

A manera de ejemplo, si `X` tiene 60x10 elementos (es decir 60 muestras con 10 características por muestra) y tiene dos clases (es decir las primeras 30 son de la primera clase y las restantes 30 de la segunda clase) entonces el comando:

`Xtrain,ytrain,Xtest,ytest = SplitTrainTest(X,y,20)`

genera la siguiente salida:

* `Xtrain` matriz de 40x10 elementos (las primeras 20 filas de la primera clase, las restante 20 filas de la segunda clase)
* `ytrain` vector de 40 elementos (20 ceros y 20 unos)
* `Xtest`  matriz de 20x10 elementos (las primeras 10 filas de la primera clase, las restante 10 filas de la segunda clase)
* `ytest`  vector de 20 elementos (10 ceros y 10 unos)


## Tercer paso: Clasificar y evaluar desempeño

Para clasificar usaremos el siguiente comando, en el que se entrena un KNN usando `Xtrain,ytrain` y se prueba en `Xtest` de la siguiente manera:

`acc,C = ClassifierKNN(Xtrain,ytrain,Xtest,ytest)`

La clasificación del conjunto de pruebas (que es almacenado internamente en la variable `ypred`) es comparado con `ytest` para calcular el Accuracy (`acc`) y la matriz de confusión (`C`).

**NOTA:** Las funciones `SplitTrainTest`  y `ClassifierKNN` se encuentran definidas más arriba en la celda "Funciones Necesarias".



In [ ]:
# Paso 1: definir X
X = Xlbp

# Paso 2: separar training/testing
ntrain = ??? # <= COMPLETAR AQUI EL NUMERO DE MUESTRAS POR CLASE PARA EL ENTRENAMIENTO
Xtrain,ytrain,Xtest,ytest = SplitTrainTest(X,y,ntrain) 

# Paso 3: clasificar y evaluar desempeño
acc,Clbp = ClassifierKNN(Xtrain,ytrain,Xtest,ytest)

In [ ]:
#
# EJECUTAR ESTA CELDA PARA EVALUAR LA CLASIFICACIÓN CON LBP
#

T[3] = CheckMatrix('Xtrain',Xtrain,chk[3]) # <Evaluación Q03>
T[4] = CheckMatrix('Xtest ',Xtest ,chk[4]) # <Evaluación Q04>
T[5] = CheckMatrix('Clbp  ',Clbp  ,chk[5]) # <Evaluación Q05>
CheckPrint(T[3:6])


# PREGUNTA 4: Clasificación usando Haralick

Repetir los tres pasos de la celda anterior para realizar una clasificación usando Haralick. 

In [ ]:
# Paso 1
X = ??? # <= COMPLETAR AQUI CON LA MATRIZ DE CARACTERÍSTICAS DE HARALICK

# Paso 2
Xtrain,ytrain,Xtest,ytest = ??? # <= COMPLETAR AQUI LA FUNCION QUE SEPARA TRAINING/TESTING

# Paso 3
acc,Char = ??? # <= COMPLETAR AQUI LA FUNCION QUE CLASIFICA Y EVALUA DESEMPEÑO

In [ ]:
#
# EJECUTAR ESTA CELDA PARA EVALUAR LA CLASIFICACIÓN CON HARALICK
#

T[6] = CheckMatrix('Xtrain',Xtrain,chk[6]) # <Evaluación Q06>
T[7] = CheckMatrix('Xtest ',Xtest ,chk[7]) # <Evaluación Q07>
T[8] = CheckMatrix('Char  ',Char  ,chk[8]) # <Evaluación Q08>
CheckPrint(T[6:9])


# PREGUNTA 5: Clasificación usando Gabor

Repetir los tres pasos de la celda anterior para realizar una clasificación usando Gabor.

In [ ]:
# Paso 1
X = ??? # <= COMPLETAR AQUI CON LA MATRIZ DE CARACTERÍSTICAS DE GABOR

# Paso 2
Xtrain,ytrain,Xtest,ytest = SplitTrainTest(X,y,ntrain) 

# Paso 3
acc,Cgab = ClassifierKNN(Xtrain,ytrain,Xtest,ytest) 

In [ ]:
#
# EJECUTAR ESTA CELDA PARA EVALUAR LA CLASIFICACIÓN CON GABOR
#

T[9]  = CheckMatrix('Xtrain',Xtrain,chk[9])    # <Evaluación Q09>
T[10] = CheckMatrix('Xtest ',Xtest ,chk[10])   # <Evaluación Q10>
T[11] = CheckMatrix('Cgab  ',Cgab  ,chk[11])   # <Evaluación Q11>
CheckPrint(T[9:12])


# PREGUNTA 6: Clasificación usando todas las características

Para este ejemplo de esta pregunta, usaremos las tres matrices `Xlbp`, `Xhar` y `Xgab`, es decir por cada muestra tendremos 59+28+67 = 154 características extraídas. Para concatenar las tres matrices, una al lado de la otra usaremos el comando `np.concatenate()` de la librería Numpy. 

A manera de ejemplo, si tenemos cuatro matrices `Z1`, `Z2`, `Z3` y `Z4`, cada una de 100x10 elementos, el comando 

`Z = np.concatenate((Z1,Z2,Z3,Z4), axis=1)`

entrega una matriz `Z` de 100x40 elementos donde

 `Z = [ Z1 | Z2 | Z3 | Z4]`.

In [ ]:
# Paso 1
X = ???  # <= COMPLETAR AQUI LA CONCATENACIÓN DE LBP, HARALICK Y GABOR (NO CAMBIAR EL ORDEN)

# Paso 2
Xtrain,ytrain,Xtest,ytest = SplitTrainTest(X,y,ntrain) 

# Paso 3
acc,Call = ClassifierKNN(Xtrain,ytrain,Xtest,ytest)

In [ ]:
#
# EJECUTAR ESTA CELDA PARA EVALUAR LA CLASIFICACIÓN CON LBP, HARALICK Y GABOR
#

T[12] = CheckMatrix('Xtrain',Xtrain,chk[12])*CheckMatrix('Xtrain',Xtrain[:,0:59],chk[3])*CheckMatrix('Xtrain',Xtrain[:,59:87],chk[6]) # <Evaluación Q12>
T[13] = CheckMatrix('Xtest ',Xtest ,chk[13])  # <Evaluación Q13>
T[14] = CheckMatrix('Call  ',Call  ,chk[14])  # <Evaluación Q14>
CheckPrint(T[12:15])


# PREGUNTA 7: Selección de características

Con el último resultado de la pregunta anterior, nos damos cuenta que al usar todas las características no necesariamente mejoramos el desempeño (ya que hay resultados individuales mejores). Una razón que puede explicar este fenómeno, es que muchas de las características extraídas no tienen un buen desempeño en estos datos porque no discriminan bien, y esto hace que el clasificador se confunda. Una solución a este problema es  seleccionar -de todas las características extraídas- aquellas que tienen un buen desempeño. En este ejemplo seleccionaremos cinco características de las 154 extraídas, que en conjunto se desempeñan muy bien en este problema. La selección fue previamente realizada y en este ejemplo está almacenada en el vector `sel`. La selección de características la veremos con detalle en el próximo capítulo del curso. 

Para seleccionar columnas se puede estudiar este ejemplo:

Si `s = [2,5,7]`, entonces con `Y = Z[:,s]` obtenemos una matriz `Y` con el mismo número de filas de `Z` y con sólo tres columnas que corresponden a la segunda, quinta y séptima columnas de `Z` respectivamente.

----
<font color='orange'> IMPORTANTE: Este ejemplo resulta bien sólo si en `X` se tienen concatenadas las matrices `Xlbp`, `Xhar` y `Xgab` (estrictamente en este orden, tal como se hizo en la pregunta anterior).

----

In [ ]:
# columnas seleccionadas con un algoritmo de selección de características:
sel = [ 28,  22,  77, 104,  65]

# Paso 1
# escoger columnas de X
Xs = ??? # <= COMPLETAR AQUI LAS COLUMNAS DE X INDEXADAS CON LA VARIABLE 'sel'

# Paso 2
# separar training / testing
Xtrain,ytrain,Xtest,ytest = SplitTrainTest(Xs,y,ntrain)

# Paso 3
# clasificar
acc,Css = ClassifierKNN(Xtrain,ytrain,Xtest,ytest)

In [ ]:
#
# EJECUTAR ESTA CELDA PARA EVALUAR LA CLASIFICACIÓN CON 5 CARACTERISTICAS SELECCIONADAS
#

T[15] = CheckMatrix('Xtrain',Xtrain,chk[15])  # <Evaluación Q15>
T[16] = CheckMatrix('Xtest ',Xtest ,chk[16])  # <Evaluación Q16>
T[17] = CheckMatrix('Css   ',Css   ,chk[17])  # <Evaluación Q17>
CheckPrint(T[15:18])


# PREGUNTA 8: Verificar datos personales

<font color='red'> **ADVERTENCIA:** En este ejercicio en necesario que revises bien tus datos personales. Si no son correctos, no tendrás nota en este ejercicio.

In [ ]:
# POR FAVOR VUELVE A DIGITAR ESTOS DATOS PARA VERIFICAR QUE NO HAYA ERRORES DE TIPEO
NumeroID_2       = '' # <= COMPLETAR AQUI NUMERO DE ALUMNO/A ENTRE APOSTROFES
Correo_2         = '' # <= COMPLETAR AQUI TU CORRECO ELECTRONICO

In [ ]:
#
# EJECUTAR ESTA CELDA PARA VERIFICAR SI LOS DATOS COINCIDEN CON LOS YA INGRESADOS
#

T[18] = NumeroID != NumeroID_2  # <Evaluación Q18>
T[19] = Correo   != Correo_2    # <Evaluación Q19>
CheckPrint(T[18:20],0)

# PASO FINAL: Revisión del Profesor




**Copyright**

The notebooks are provided as [Open Educational Resource](https://de.wikipedia.org/wiki/Open_Educational_Resources). Feel free to use the notebooks for your own educational purposes. The text is licensed under [Creative Commons Attribution 4.0](https://creativecommons.org/licenses/by/4.0/), the code of the IPython examples under the [MIT license](https://opensource.org/licenses/MIT).